In [1]:
import sqlite3
from keras.models import load_model
from matplotlib.pyplot import imshow
import tqdm.notebook as tqdm
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.spatial import distance
import time
import uuid
import json
from imageio import imread
import random

from multiprocessing import Pool, Process

import pickle
import mysql.connector

import numpy as np

from FaceLoader import FaceLoader
from SocialProcessor import SocialProcessor

import vk_api

%matplotlib inline

Using TensorFlow backend.


In [2]:
conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")

In [3]:
model_path = 'keras-facenet-master/model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def aunt_vk():
    return input(), True
def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [5]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [6]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [7]:
with open('vk_already.pcl', 'rb') as file:
    ids_already = pickle.load(file)

In [8]:
ids_scrape = set([185313244])
ids_scrape = ids_scrape - ids_already

In [9]:
ids_already

set()

In [10]:
processor = SocialProcessor(conn, model, limit=20)

In [11]:
add_set = set()
while True:
    with open('vk_already.pcl', 'rb') as file:
        ids_already = pickle.load(file)
        
    ids_scrape = ids_scrape - ids_already
    g_list = list(ids_scrape)
    random.shuffle(g_list)
    i = g_list[0]
    
    print("To be processed:", len(g_list))
    print("Deeper level:", len(add_set))
    print(i, ':')
    
    processor.processVkUser(api, i)
    ids_already.add(i)
    with open('vk_already.pcl', 'wb') as file:
        pickle.dump(ids_already, file)

    try:
        add_set = add_set.union(set(api.friends.get(user_id=i)['items']))
    except:
        pass
    
    ids_scrape = ids_scrape - ids_already
    if len(ids_scrape) == 0:
        ids_scrape = ids_scrape.union(add_set)
        add_set = set()

To be processed: 1
Deeper level: 0
185313244 :


100%|██████████| 20/20 [00:05<00:00,  3.65it/s]


Added 25 rows
To be processed: 235
Deeper level: 0
209470691 :


 65%|██████▌   | 13/20 [00:06<00:03,  2.17it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/aleksandrdremov/PycharmProjects/GlobalScraper/FaceLoader.py", line 71, in load_and_align_image
    img = imread(self.local_url)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/imageio/core/functions.py", line 265, in imread
    reader = read(uri, format, "i", **kwargs)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/imageio/core/functions.py", line 172, in get_reader
    request = Request(uri, "r" + mode, **kwargs)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/imageio/core/request.py", line 124, in __init__
    self._parse_uri(uri)
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/imageio/core/request.py", line 260, in _parse_uri
    raise FileNotFoundError("No such file: '%s'" % fn)
FileNotFoundError: No such file: '/Users/aleksandrdremov/PycharmProjects/GlobalScraper/tmp/4bb7e6642d3364129f

TypeError: object of type 'NoneType' has no len()

In [ ]:
with open('vk_already.pcl', 'wb') as file:
        pickle.dump(ids_already, file)